<a href="https://colab.research.google.com/github/IsuruMahakumara/microsoft-ai-ml-engineering/blob/main/Practice_activity_Model_and_dataset_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Loading Bert for Text Classification

num_labels=3: this parameter specifies the number of labels (or classes) the model will classify into. For example, in a sentiment analysis task, num_labels=2 might be used for binary classification (positive/negative), while a task with more categories would require a higher value.

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT model and tokenizer for classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Model and tokenizer are now ready for fine-tuning

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Task Specific Datasets


*   [Kaggle Datasets](https://www.kaggle.com/datasets)
*   [Google Dataset Search](https://datasetsearch.research.google.com/)
*  [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)





Hugging Face Datasets provide built-in support for data transformations and augmentations. You can explore their documentation for augmentation techniques:
Hugging Face Dataset Documentation


nlpaug is a popular Python library for augmenting text datasets using such methods as paraphrasing, backtranslation, and synonym replacement. You can find the documentation and examples here:
nlpaug Documentation



In [6]:
!pip install nlpaug

!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.1 MB/s eta 0:00:00


In [7]:
from nlpaug.augmenter.word import BackTranslationAug

# Initialize the backtranslation augmenter (English -> French -> English)
back_translation_aug = BackTranslationAug(from_model_name='facebook/wmt19-en-de', to_model_name='facebook/wmt19-de-en')

# Example text to augment
text = "The weather is great today."

# Perform backtranslation to create augmented text
augmented_text = back_translation_aug.augment(text)

print("Original text:", text)
print("Augmented text:", augmented_text)

tokenizer_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

vocab-src.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Original text: The weather is great today.
Augmented text: ['The weather today is great.']


In [10]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the IMDB movie reviews dataset for sentiment analysis
dataset = load_dataset('imdb')

# # Split the dataset into training and validation sets (if not presplit)
# train_data, val_data = train_test_split(dataset['train'], test_size=0.2)


train_test_split_dataset = dataset['train'].train_test_split(test_size=0.2)

# Extract the new training and validation splits
train_data = train_test_split_dataset['train']
val_data = train_test_split_dataset['test']




# Convert the data into the format required for tokenization
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [12]:
# Tokenize the dataset
tokenized_train = tokenizer(
    list(train_data['text']), # FIX: Explicitly convert the column to a Python list
    padding=True,
    truncation=True,
    return_tensors="pt"
)
tokenized_val = tokenizer(
    list(val_data['text']),   # FIX: Explicitly convert the column to a Python list
    padding=True,
    truncation=True,
    return_tensors="pt"
)